# Run PEST and Parallel PEST
### This should be super easy ~

In [2]:
import os
import shutil
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt;
import psutil
import sys
import pyemu
import flopy
import swatp_pst
assert "dependencies" in flopy.__file__
assert "dependencies" in pyemu.__file__
assert "dependencies" in swatp_pst.__file__

# 01. Specify a path to the model folder

In [3]:
# path to project directory
prj_dir = "D:\\spark\\jj\\opt_2nd" # win


In [4]:
main_opt_path = os.path.join(prj_dir, 'main_opt')
os.chdir(main_opt_path)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:\\spark\\jj\\opt_2nd\\main_opt'

In [ ]:
os.getcwd()

# 02. Perform GSA

In [25]:
# pst_sen = pyemu.Pst(os.path.join(main_opt_path,"swatp_nw_ies.pst"))
# pst_sen.write('swatp_nw_sen.pst', version=2) # write new IES control file

In [26]:
os.getcwd()

'D:\\spark\\jj\\opt_2nd\\main_opt'

In [31]:
pst_sen2 = pyemu.Pst(os.path.join(main_opt_path,"swatp_nw_ies.pst"))
# pst_sen2.control_data.formatted_values
pst_sen2.pestpp_options["gsa_method"] = "sobol"
pst_sen2.pestpp_options["gsa_sobol_samples"] = 200
pst_sen2.pestpp_options["gsa_sobol_par_dist"] = "unif"
# pst_sen2.pestpp_options['ies_num_reals'] = 200 # number of realization

pst_sen2.write('swatp_nw_sen_sobol.pst', version=2) # write new IES control file

noptmax:10, npar_adj:10, nnz_obs:124


In [7]:
# set a path to the main directory of model
m_d = os.path.join(prj_dir, "swatp_nw_sen_sobol")

In [ ]:
os.chdir(prj_dir) # move the loction to project directory
# run PEST in parallel
pyemu.os_utils.start_workers(
                            main_opt_path, # the folder which contains the "template" PEST dataset
                            "pestpp-sen", #the PEST software version we want to run
                            'swatp_nw_sen_sobol.pst', # the control file to use with PEST
                            num_workers=num_workers, #how many agents to deploy
                            worker_root='.', #where to deploy the agent directories; relative to where python is running
                            master_dir=m_d, #the manager directory,
                            # reuse_master=True
                            )

# 03. Let's see how we did:

In [8]:
pst_name = 'swatp_nw_sen.pst'

In [9]:
os.chdir(m_d)
df = pd.read_csv(os.path.join(m_d,pst_name.replace(".pst",".msn")), index_col='parameter_name')
df.head()

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:\\spark\\jj\\opt_2nd\\swatp_nw_sen_sobol'

In [1]:

df = df.loc[df.sen_mean_abs>1e-6,:]
df.loc[:,["sen_mean_abs","sen_std_dev"]].plot(kind="bar", figsize=(13,4))
#ax = plt.gca()
#ax.set_ylim(1,ax.get_ylim()[1]*1.1)
plt.yscale('log');
fig,ax = plt.subplots(1,1,figsize=(13,8))
tmp_df = df
ax.scatter(tmp_df.sen_mean_abs,tmp_df.sen_std_dev,marker="^",s=20,c="r")
tmp_df = tmp_df.iloc[:8]
for x,y,n in zip(tmp_df.sen_mean_abs,tmp_df.sen_std_dev,tmp_df.index):
    ax.text(x,y,n)
mx = max(ax.get_xlim()[1],ax.get_ylim()[1])
mn = min(ax.get_xlim()[0],ax.get_ylim()[0])
ax.plot([mn,mx],[mn,mx],"k--")
ax.set_ylim(mn,mx)
ax.set_xlim(mn,mx)
ax.grid()
ax.set_ylabel("σ")
ax.set_xlabel("μ")
ax.tick_params(axis='both', labelsize=12)
plt.legend(
    fontsize=12, 
#     loc="lower left"
)
plt.tight_layout()

NameError: name 'df' is not defined

In [ ]:
# plot 1 to 1
figs = pst.plot(kind="1to1")

In [ ]:
# plot phi progress
pyemu.plot_utils.phi_progress(pst)